## Importing Libraries

In [1]:
from sklearn.model_selection import GridSearchCV
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import requests
from datetime import datetime, timedelta


## Creating Prediction Model Based on Data Set

In [2]:
file_paths = {
    'New York': 'Data/New York City 2021-07-01 to 2024-03-24.csv',
    'Chicago': 'Data/Chicago 2021-07-01 to 2024-03-17.csv',
    'Austin': 'Data/Bergstrom International A... 2021-07-01 to 2024-03-17.csv',
    'Miami': 'Data/Miami, Florida 2021-07-01 to 2024-03-17.csv'
}

# Load and combine datasets with an additional 'Location' column
dataframes = []
for location, file_path in file_paths.items():
    df = pd.read_csv(file_path)
    df['Location'] = location  # Add the location as a feature
    dataframes.append(df)
combined_data = pd.concat(dataframes, ignore_index=True)

combined_data['datetime'] = pd.to_datetime(combined_data['datetime'])
combined_data['Year'] = combined_data['datetime'].dt.year
combined_data['Month'] = combined_data['datetime'].dt.month
combined_data['Day'] = combined_data['datetime'].dt.day
combined_data['DayOfYear'] = combined_data['datetime'].dt.dayofyear
combined_data['WeekOfYear'] = combined_data['datetime'].dt.isocalendar().week
combined_data['windgust'] = combined_data['windgust'].fillna(0)

# Feature selection
features_to_include = ['humidity', 'windgust',
                       'cloudcover', 'tempmin', 'tempmax']
X = combined_data[['Year', 'Month', 'Day', 'DayOfYear',
          'WeekOfYear'] + features_to_include]
y = combined_data['tempmax']

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

# Define and train the model
optimized_model = XGBRegressor(
    n_estimators=656,
    max_depth=5,
    learning_rate=0.24057515827045411,
    colsample_bytree=0.9500983814768778,
    subsample=0.27848270266263386,
    reg_alpha=0.3728777359951313,
    reg_lambda=1.9881262022259722,
    random_state=42,
    early_stopping_rounds=50  # Moved here
)

# optimized_model = XGBRegressor(
#     n_estimators=656,
#     max_depth=5,
#     learning_rate=0.24057515827045411,
#     colsample_bytree=0.9500983814768778,
#     subsample=0.27848270266263386,
#     reg_alpha=0.3728777359951313,
#     reg_lambda=1.9881262022259722,
#     random_state=42
# )
optimized_model.fit(X_train, y_train, eval_set=[
                    (X_val, y_val)], verbose=False)




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9500983814768778, device=None,
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.24057515827045411, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=656, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

## Predicting today's max temperature

In [5]:
API_KEY = 'TTLKP6SD4U78PMAB9UHBVLBYH'
locations = ['New York', 'Chicago', 'Austin', 'Miami']
unit_group = 'us'  # or 'metric' for Celsius

df_weather = pd.DataFrame()
previous_day_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

for location in locations:
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{previous_day_date}?unitGroup={unit_group}&key={API_KEY}&include=obs'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        previous_day_data = data['days'][0]
        
        temp_df = pd.DataFrame([{
            'Location': location,
            'humidity': previous_day_data.get('humidity'),
            'windgust': previous_day_data.get('windspeed', 0),
            'cloudcover': previous_day_data.get('cloudcover'),
            'tempmin': previous_day_data.get('tempmin'),
            'tempmax':previous_day_data.get('tempmax'),
            'Year': int(previous_day_date.split('-')[0]),
            'Month': int(previous_day_date.split('-')[1]),
            'Day': int(previous_day_date.split('-')[2]),
        }])

        
        
        df_weather = pd.concat([df_weather, temp_df], ignore_index=True)
    else:
        print(f"Failed to retrieve data for {location}")
# If these were part of the original training data and hence expected by the scaler
df_weather['DayOfYear'] = pd.to_datetime(df_weather[['Year', 'Month', 'Day']]).dt.dayofyear
df_weather['WeekOfYear'] = pd.to_datetime(df_weather[['Year', 'Month', 'Day']]).dt.isocalendar().week

# Make sure 'features_to_include' includes all features the scaler expects
features_to_include = ['humidity', 'windgust', 'cloudcover', 'tempmin', 'tempmax']
X_weather = df_weather[['Year', 'Month', 'Day', 'DayOfYear',
          'WeekOfYear']+features_to_include]

# Transform features with the pre-fitted scaler
X_weather_scaled = scaler.transform(X_weather)  # Assuming 'scaler' is already fitted

# Make predictions using the trained model
df_weather['predicted_tempmax'] = optimized_model.predict(X_weather_scaled)
# Round the 'predicted_tempmax' column to 0 decimal places for whole numbers
df_weather['predicted_tempmax'] = df_weather['predicted_tempmax'].round(0)

# Specify the file name for the CSV
output_csv_file = 'predicted_temperature_max.csv'

# Save the DataFrame to CSV, without the index
df_weather[['Location', 'predicted_tempmax']].to_csv(output_csv_file, index=False)

# Now, print the DataFrame with the 'Location' and rounded 'predicted_tempmax'

print(df_weather[['Location', 'predicted_tempmax']])


   Location  predicted_tempmax
0  New York               49.0
1   Chicago               51.0
2    Austin               75.0
3     Miami               84.0
